In [1]:
# import sys
# import os
# project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
# sys.path.append(project_root)

from coolprompt.assistant import PromptTuner

/home/nikita.kulin/CoolPrompt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Basic tutorial of CoolPrompt
*Actual for 1.1.0+ version*

## Quick Run

In [ ]:
from coolprompt.assistant import PromptTuner

tuner = PromptTuner()

tuner.run('Write an essay about autumn')

In [4]:
print(tuner.final_prompt)


You are an expert writer and seasonal observer tasked with composing a rich, well-structured, and vividly descriptive essay on the theme of autumn. Your goal is to create a coherent, engaging, and insightful piece that captures the essence of the season through sensory detail, natural observation, and cultural reflection.

Begin by describing the key physical and environmental characteristics of autumn: the transformation of foliage, the gradual cooling of temperatures, the crispness of the air, and the quiet rhythm of nature as leaves fall. Use specific, evocative language to convey sight, sound, smell, and touch—such as the rustle of dry leaves, the golden light filtering through trees, or the earthy scent of damp soil after rain.

Integrate cultural and human elements of autumn, including harvest festivals (e.g., Thanksgiving, Halloween, harvest fairs), traditional foods (e.g., apples, pumpkins, squash), and seasonal rituals that reflect humanity’s connection to nature. Reflect on 

### What happened step-by-step
1. We initialized a CoolPrompt Tuner with default large language model: qwen3-4B-instruct
2. We input a start prompt that should be modified
3. CoolPrompt auto-detected preferred task and metric for prompt optimization and evaluation
4. CoolPrompt PromptAssistant generated automatically the dataset with target labels and split into train and test samples
5. The default prompt optimizer HyPE suggested the optimized prompt
6. CoolPrompt outputs the results:
   - Metric scores of optimization efficiency
   - The final prompt
   - The interpretation of prompt optimization

### That is a simple approach how to optimize prompts. Let's try to configure CoolPrompt Tuner

## Setup a CoolPrompt Tuner

### 1. LLM Choice
The framework is a model-agnostic, you can use proprietary, open-source or custom LLMs with different interfaces via LangChain compatibility. 

#### Check [the full list of provider interfaces](https://python.langchain.com/docs/integrations/llms/).

Default LLM in CoolPrompt is Qwen/Qwen3-4B-Instruct-2507 and it a same can be defined as:

```python
llm = HuggingFacePipeline.from_model_id(
    model_id="Qwen/Qwen3-4B-Instruct-2507",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 4000,
        "temperature": 0.01,
        "do_sample": False,
        "return_full_text": False,
    }
)
target_model = ChatHuggingFace(llm=llm)

prompt_tuner = PromptTuner(target_model=target_model)
```


You can use other popular interfaces as:

#### Ollama
For rapid experiments with low resources

In [ ]:
from langchain_ollama.llms import OllamaLLM

my_model = OllamaLLM(
    model="qwen2.5-coder:32b"
)
prompt_tuner = PromptTuner(target_model=my_model)

#### VLLM
As a production ready solution

In [ ]:
from langchain_community.llms import VLLM

my_model = VLLM(
    model="Qwen/Qwen3-4B-Instruct-2507",
    trust_remote_code=True,
    dtype='bfloat16',
)

prompt_tuner = PromptTuner(target_model=my_model)

#### ChatOpenAI
For OpenAI and OpenAI compatible models

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

my_model = ChatOpenAI(
    model="paste model_name",
    base_url="paste base_url",
    openai_api_key="paste api_key",
    temperature=0.01,
    max_tokens=4000,
)

prompt_tuner = PromptTuner(target_model=my_model)

### 2. System Model
Argument `system_model` defines a core in a **PromptAssistant** module.

**PromptAssistant** is responsible for:
- Definition automatically a task type and a metric for the specific start prompt.
- Generates a syntetic dataset and target labels (when the real data is not provided).
- Provides an optimization feedback.

By default a `system_model` is a defined the same as a `target_model`. It could be set with a different llm in code below:

In [ ]:
from langchain_community.llms import VLLM

target_model = VLLM(
    model="Qwen/Qwen3-4B-Instruct-2507"
)
system_model = VLLM(
    model="Qwen/Qwen3-30B-A3B-Instruct-2507"
)

prompt_tuner = PromptTuner(
    target_model=target_model,
    system_model=system_model
)

#### IMPORTANT NOTE: `system_model` needs to be a confident instructional llm that can generate a structual output.

### 3. Task and metrics
CoolPrompt supports 2 task types and a set of metrics: 
- `classification` - a common classification
    - `accuracy`
    - `f1` (f1-macro)
- `generation` - a general new text generation
    - `bleu`
    - `rouge`
    - `meteor`
    - `bertscore`
    - `geval` (one metric: *textual accuracy*)

For each run you can define as follows:

In [ ]:
prompt_tuner.run(
    start_prompt="Classify a sentence sentiment",
    task="classification",
    metric="f1",
)

prompt_tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    metric="rouge",
)

### 4. Setup a dataset configuration

#### 4.1 Input a real dataset

In [ ]:
from datasets import load_dataset

sst2 = load_dataset("stanfordnlp/sst2")
class_dataset = sst2['train']['sentence'][:100]
class_targets = sst2['train']['label'][:100]

tuner.run(
    start_prompt="Classify sentence sentiment",
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="f1"
)

In [ ]:
samsum = load_dataset("knkarthick/samsum")
gen_dataset = samsum['train']['dialogue'][:100]
gen_targets = samsum['train']['summary'][:100]

tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="bertscore"
)

#### 4.2 Define a split strategy

`validation_size` - a ratio for train-valid split

In [ ]:
tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="bertscore",
    validation_size=0.4
)

`train_as_test` - assign a test set as a train, `validation_size` will be ignored

In [ ]:
tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="bertscore",
    train_as_test=True
)

### 5. Choosing an optimizer

#### HyPE

In [ ]:
tuner.run(
    start_prompt="Summarize the text",
    method='hype'
)

#### ReflectivePrompt

In [ ]:
tuner.run(
    start_prompt="Summarize the text",
    method='reflective'
)

#### DistillPrompt

In [ ]:
tuner.run(
    start_prompt="Summarize the text",
    method='distill'
)

## Framework Results

#### Metrics before-after

In [ ]:
print('Before optimization:', tuner.init_metric)
print('After optimization:', tuner.final_metric)

Before optimization: 0.3997358145852188
After optimization: 0.3607928178068911


#### Prompts before-after

In [ ]:
print('Before optimization:', tuner.init_prompt)
print('After optimization:', tuner.final_prompt)

Before optimization: Write an essay about autumn
After optimization: 
You are an expert writer and seasonal observer tasked with composing a rich, well-structured, and vividly descriptive essay on the theme of autumn. Your goal is to create a coherent, engaging, and insightful piece that captures the essence of the season through sensory detail, natural observation, and cultural reflection.

Begin by describing the key physical and environmental characteristics of autumn: the transformation of foliage, the gradual cooling of temperatures, the crispness of the air, and the quiet rhythm of nature as leaves fall. Use specific, evocative language to convey sight, sound, smell, and touch—such as the rustle of dry leaves, the golden light filtering through trees, or the earthy scent of damp soil after rain.

Integrate cultural and human elements of autumn, including harvest festivals (e.g., Thanksgiving, Halloween, harvest fairs), traditional foods (e.g., apples, pumpkins, squash), and seaso

#### Generated dataset

In [ ]:
print(tuner.synthetic_dataset)
print(tuner.synthetic_target)

#### Optimization feedback

In [ ]:
print('Feedback', tuner.assistant_feedback)